In [2]:
from models import SigRep
from CONSTANT import *
from tools import *
import pandas as pd
import numpy as np
from torch.utils.data import (TensorDataset, DataLoader, SequentialSampler, WeightedRandomSampler)

import torch
import torch.nn.functional as F

from torch import nn

use_cuda = False
valid='loso'

device = torch.device('cuda' if torch.cuda.is_available() and use_cuda else 'cpu')
print('using', device)

using cpu


In [3]:
class Params(object):
    def __init__(self, lr=0.001, epoch=100, valid='loso', target='valence', batch_size=64,):
        self.batch_size = batch_size
        self.valid = valid
        self.target = target
        self.epoch = epoch
        self.lr = lr
        self.metrics_dict = {}
        self.debug = True
        self.out_channels = 32
        self.out_channels = 32
        self.hidden_size = 64
        self.num_layers = 1
        self.fcn_input = 50432

In [4]:
spliter = load_model(r'./processed_signal/WESAD/400_4s_step_2s_spliter.pkl')
data = pd.read_pickle(r'./processed_signal/WESAD/400_4s_step_2s.pkl')
# data = pd.read_csv(r'./processed_signal/HKU956/400_4s_step_2s.csv')

for k in spliter[valid]:
    train_index = k['train_index']
    test_index = k['test_index']
    break

In [6]:
args = Params()
dataprepare = DataPrepare(args=args, target='valence_rating', data=data, train_index=train_index, test_index=test_index, device=device)
train_dataloader, test_dataloader = dataprepare.get_data()

(1000, 4, 400) (1000, 1) (100, 4, 400) (100, 1)


In [8]:
batch_size = 32
learning_rate = 0.1

num_epochs = 10

model = SigRep.SigRepSimple(4)

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
use_gpu = False

# 开始模型训练
for epoch in range(num_epochs):
    print('*' * 10)
    print(f'epoch {epoch+1}')
    running_loss = 0.0 # 初始值
    running_acc = 0.0
    for i, data in enumerate(train_dataloader, 1): # 枚举函数enumerate返回下标和值
        img, label = data
        print(label)
        print(label.shape, label.type())
        img = img.view(img.size(0), -1) # 将图片展开为28*28
        # 使用GPU？
        if use_gpu:
            img = img.cuda()
            label = label.cuda()
        # 向前传播
        out = model(img) # 前向传播
        loss = criterion(out, label) # 计算loss
        running_loss += loss.item() # loss求和
        _, pred = torch.max(out, 1)
        running_acc += (pred == label).float().mean()
        # 向后传播
        optimizer.zero_grad() # 梯度归零
        loss.backward() # 后向传播
        optimizer.step() # 更新参数

        if i % 300 == 0:
            print(f'[{epoch+1}/{num_epochs}] Loss: {running_loss/i:.6f}, Acc: {running_acc/i:.6f}')
    print(f'Finish {epoch+1} epoch, Loss: {running_loss/i:.6f}, Acc: {running_acc/i:.6f}')

**********
epoch 1
tensor([[6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.],
        [6.]])
torch.Size([64, 1]) torch.FloatTensor


IndexError: too many indices for tensor of dimension 2